Este código implementa una GAN básica usando Keras para generar imágenes de dígitos similares a los de MNIST. El generador aprende a crear imágenes que se asemejan a las imágenes reales, mientras que el discriminador aprende a distinguir entre imágenes reales y falsas. Durante el entrenamiento, estos dos modelos se entrenan conjuntamente hasta que el generador produce imágenes realistas.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input, Model


#### Cargar y preprocesar los datos MNIST

In [2]:
# Cargar el conjunto de datos MNIST
(X_train, _), (_, _) = mnist.load_data()

# Normalizar los datos
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

#### Construir el generador

In [3]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))

    return model

#### Construir el discriminador

In [4]:
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [5]:
#### Compilar los modelos

In [6]:
img_shape = (28, 28, 1)
latent_dim = 100

# Crear y compilar el discriminador
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Crear el generador
generator = build_generator(latent_dim)

# El generador toma ruido como entrada y genera imágenes
z = Input(shape=(latent_dim,))
img = generator(z)

# Para el modelo combinado, solo entrenamos el generador
discriminator.trainable = False

# El discriminador toma imágenes generadas como entrada y determina su validez
valid = discriminator(img)

# Modelo combinado (stacked generador y discriminador)
# Entrenamos el generador para engañar al discriminador
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

In [7]:
#### Entrenar la GAN

In [8]:
def train(epochs, batch_size=128, save_interval=50):
    # Cargar los datos
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Etiquetas para imágenes reales y falsas
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Entrenar el discriminador

        # Seleccionar un conjunto aleatorio de imágenes reales
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Generar un conjunto de imágenes falsas
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Entrenar el discriminador
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenar el generador

        # Generar un nuevo conjunto de ruido
        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Entrenar el generador para engañar al discriminador
        g_loss = combined.train_on_batch(noise, valid)

        # Imprimir el progreso
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        # Guardar imágenes generadas a intervalos
        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Escalar las imágenes generadas de [-1, 1] a [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"mnist_{epoch}.png")
    plt.close()

# Entrenar la GAN
train(epochs=10000, batch_size=64, save_interval=200)

2/2 [==============================] - 1s 34ms/step
0 [D loss: 0.73634472489357 | D accuracy: 59.375] [G loss: 1.0544862747192383]
2/2 [==============================] - 0s 10ms/step
1 [D loss: 0.34707261249423027 | D accuracy: 92.1875] [G loss: 1.0641224384307861]
2/2 [==============================] - 0s 10ms/step
2 [D loss: 0.31637278012931347 | D accuracy: 93.75] [G loss: 1.0490102767944336]
2/2 [==============================] - 0s 10ms/step
3 [D loss: 0.3051759898662567 | D accuracy: 93.75] [G loss: 1.1121134757995605]
2/2 [==============================] - 0s 10ms/step
4 [D loss: 0.2779234014451504 | D accuracy: 98.4375] [G loss: 1.242950201034546]
2/2 [==============================] - 0s 0s/step
5 [D loss: 0.2593770707026124 | D accuracy: 96.09375] [G loss: 1.339322805404663]
2/2 [==============================] - 0s 10ms/step
6 [D loss: 0.22574764117598534 | D accuracy: 98.4375] [G loss: 1.4956690073013306]
2/2 [==============================] - 0s 0s/step
7 [D loss: 0.185150

2/2 [==============================] - 0s 0s/step
123 [D loss: 0.006578441709280014 | D accuracy: 100.0] [G loss: 5.003703594207764]
2/2 [==============================] - 0s 8ms/step
124 [D loss: 0.01635706052184105 | D accuracy: 100.0] [G loss: 5.011531829833984]
2/2 [==============================] - 0s 8ms/step
125 [D loss: 0.00848797569051385 | D accuracy: 100.0] [G loss: 4.982240676879883]
2/2 [==============================] - 0s 0s/step
126 [D loss: 0.011466403491795063 | D accuracy: 100.0] [G loss: 4.992547035217285]
2/2 [==============================] - 0s 10ms/step
127 [D loss: 0.01613406289834529 | D accuracy: 100.0] [G loss: 5.041369915008545]
2/2 [==============================] - 0s 10ms/step
128 [D loss: 0.012254089349880815 | D accuracy: 100.0] [G loss: 5.224635124206543]
2/2 [==============================] - 0s 10ms/step
129 [D loss: 0.008922290056943893 | D accuracy: 100.0] [G loss: 5.039366722106934]
2/2 [==============================] - 0s 8ms/step
130 [D loss: 

2/2 [==============================] - 0s 10ms/step
244 [D loss: 0.07050340250134468 | D accuracy: 100.0] [G loss: 3.9030237197875977]
2/2 [==============================] - 0s 0s/step
245 [D loss: 0.22971925884485245 | D accuracy: 91.40625] [G loss: 2.6610641479492188]
2/2 [==============================] - 0s 2ms/step
246 [D loss: 0.14033173583447933 | D accuracy: 94.53125] [G loss: 3.7481017112731934]
2/2 [==============================] - 0s 8ms/step
247 [D loss: 0.15265259891748428 | D accuracy: 96.875] [G loss: 3.567262649536133]
2/2 [==============================] - 0s 0s/step
248 [D loss: 0.33029650151729584 | D accuracy: 82.8125] [G loss: 3.03163480758667]
2/2 [==============================] - 0s 0s/step
249 [D loss: 0.10628557950258255 | D accuracy: 96.875] [G loss: 4.277101993560791]
2/2 [==============================] - 0s 5ms/step
250 [D loss: 0.6801993250846863 | D accuracy: 73.4375] [G loss: 2.177187442779541]
2/2 [==============================] - 0s 0s/step
251 [D l

2/2 [==============================] - 0s 8ms/step
305 [D loss: 0.3938983380794525 | D accuracy: 82.8125] [G loss: 2.4394326210021973]
2/2 [==============================] - 0s 8ms/step
306 [D loss: 0.13585815578699112 | D accuracy: 97.65625] [G loss: 3.2000069618225098]
2/2 [==============================] - 0s 0s/step
307 [D loss: 0.43566083908081055 | D accuracy: 80.46875] [G loss: 3.2158050537109375]
2/2 [==============================] - 0s 10ms/step
308 [D loss: 0.3949599713087082 | D accuracy: 84.375] [G loss: 3.020915985107422]
2/2 [==============================] - 0s 8ms/step
309 [D loss: 0.5520078390836716 | D accuracy: 71.09375] [G loss: 2.5478034019470215]
2/2 [==============================] - 0s 2ms/step
310 [D loss: 0.22415708750486374 | D accuracy: 94.53125] [G loss: 2.993239641189575]
2/2 [==============================] - 0s 0s/step
311 [D loss: 0.7432408630847931 | D accuracy: 63.28125] [G loss: 1.7181264162063599]
2/2 [==============================] - 0s 10ms/step

2/2 [==============================] - 0s 5ms/step
366 [D loss: 0.44223424792289734 | D accuracy: 78.125] [G loss: 2.6131885051727295]
2/2 [==============================] - 0s 4ms/step
367 [D loss: 0.6170396208763123 | D accuracy: 61.71875] [G loss: 1.8403832912445068]
2/2 [==============================] - 0s 4ms/step
368 [D loss: 0.33230800926685333 | D accuracy: 82.8125] [G loss: 2.792707920074463]
2/2 [==============================] - 0s 5ms/step
369 [D loss: 0.7551425695419312 | D accuracy: 55.46875] [G loss: 1.565031886100769]
2/2 [==============================] - 0s 4ms/step
370 [D loss: 0.336686909198761 | D accuracy: 85.9375] [G loss: 2.852360486984253]
2/2 [==============================] - 0s 5ms/step
371 [D loss: 0.6648549735546112 | D accuracy: 60.15625] [G loss: 2.058487892150879]
2/2 [==============================] - 0s 4ms/step
372 [D loss: 0.3311769962310791 | D accuracy: 88.28125] [G loss: 3.186457633972168]
2/2 [==============================] - 0s 0s/step
373 [D

...

2/2 [==============================] - 0s 0s/step
9503 [D loss: 0.6747272610664368 | D accuracy: 58.59375] [G loss: 0.8295377492904663]
2/2 [==============================] - 0s 2ms/step
9504 [D loss: 0.657940149307251 | D accuracy: 57.03125] [G loss: 0.9477214813232422]
2/2 [==============================] - 0s 0s/step
9505 [D loss: 0.6271986961364746 | D accuracy: 67.1875] [G loss: 0.8893101215362549]
2/2 [==============================] - 0s 0s/step
9506 [D loss: 0.6587070524692535 | D accuracy: 56.25] [G loss: 0.8903231620788574]
2/2 [==============================] - 0s 2ms/step
9507 [D loss: 0.6998604834079742 | D accuracy: 55.46875] [G loss: 0.9221161603927612]
2/2 [==============================] - 0s 10ms/step
9508 [D loss: 0.6518131494522095 | D accuracy: 57.8125] [G loss: 0.9827781915664673]
2/2 [==============================] - 0s 2ms/step
9509 [D loss: 0.6713716387748718 | D accuracy: 58.59375] [G loss: 0.9316175580024719]
2/2 [==============================] - 0s 0s/step

2/2 [==============================] - 0s 8ms/step
9623 [D loss: 0.7011589109897614 | D accuracy: 54.6875] [G loss: 0.910332977771759]
2/2 [==============================] - 0s 4ms/step
9624 [D loss: 0.6488593816757202 | D accuracy: 63.28125] [G loss: 0.9038844108581543]
2/2 [==============================] - 0s 0s/step
9625 [D loss: 0.6777088344097137 | D accuracy: 54.6875] [G loss: 0.8856533765792847]
2/2 [==============================] - 0s 0s/step
9626 [D loss: 0.6295451521873474 | D accuracy: 66.40625] [G loss: 0.8740314245223999]
2/2 [==============================] - 0s 0s/step
9627 [D loss: 0.6559324562549591 | D accuracy: 60.9375] [G loss: 0.9314213991165161]
2/2 [==============================] - 0s 0s/step
9628 [D loss: 0.7032551169395447 | D accuracy: 54.6875] [G loss: 0.9037591814994812]
2/2 [==============================] - 0s 0s/step
9629 [D loss: 0.6646635830402374 | D accuracy: 60.9375] [G loss: 0.9291108250617981]
2/2 [==============================] - 0s 0s/step
96

2/2 [==============================] - 0s 8ms/step
9744 [D loss: 0.684125155210495 | D accuracy: 57.03125] [G loss: 0.8835140466690063]
2/2 [==============================] - 0s 5ms/step
9745 [D loss: 0.6758444309234619 | D accuracy: 58.59375] [G loss: 0.9022447466850281]
2/2 [==============================] - 0s 8ms/step
9746 [D loss: 0.6847509443759918 | D accuracy: 60.9375] [G loss: 0.8659389019012451]
2/2 [==============================] - 0s 8ms/step
9747 [D loss: 0.6566050052642822 | D accuracy: 61.71875] [G loss: 0.9028497934341431]
2/2 [==============================] - 0s 8ms/step
9748 [D loss: 0.6458742022514343 | D accuracy: 60.9375] [G loss: 0.9297634363174438]
2/2 [==============================] - 0s 0s/step
9749 [D loss: 0.6785633265972137 | D accuracy: 58.59375] [G loss: 0.8804170489311218]
2/2 [==============================] - 0s 0s/step
9750 [D loss: 0.6469879448413849 | D accuracy: 66.40625] [G loss: 0.8441519737243652]
2/2 [==============================] - 0s 0s/s

9864 [D loss: 0.6868541240692139 | D accuracy: 55.46875] [G loss: 0.9632187485694885]
2/2 [==============================] - 0s 2ms/step
9865 [D loss: 0.6704413592815399 | D accuracy: 57.03125] [G loss: 0.9419487118721008]
2/2 [==============================] - 0s 5ms/step
9866 [D loss: 0.6505627036094666 | D accuracy: 61.71875] [G loss: 0.8641372919082642]
2/2 [==============================] - 0s 10ms/step
9867 [D loss: 0.6551546454429626 | D accuracy: 62.5] [G loss: 0.9216580390930176]
2/2 [==============================] - 0s 0s/step
9868 [D loss: 0.6458539664745331 | D accuracy: 63.28125] [G loss: 0.8710982203483582]
2/2 [==============================] - 0s 10ms/step
9869 [D loss: 0.6532167196273804 | D accuracy: 60.15625] [G loss: 0.9787864089012146]
2/2 [==============================] - 0s 10ms/step
9870 [D loss: 0.6221267879009247 | D accuracy: 60.15625] [G loss: 0.862973690032959]
2/2 [==============================] - 0s 10ms/step
9871 [D loss: 0.6814990043640137 | D accura

2/2 [==============================] - 0s 5ms/step
9925 [D loss: 0.6678287982940674 | D accuracy: 60.9375] [G loss: 0.8735325336456299]
2/2 [==============================] - 0s 0s/step
9926 [D loss: 0.6342677474021912 | D accuracy: 64.0625] [G loss: 0.9292410016059875]
2/2 [==============================] - 0s 8ms/step
9927 [D loss: 0.623054563999176 | D accuracy: 67.1875] [G loss: 0.879671573638916]
2/2 [==============================] - 0s 0s/step
9928 [D loss: 0.658875048160553 | D accuracy: 56.25] [G loss: 0.9027432203292847]
2/2 [==============================] - 0s 0s/step
9929 [D loss: 0.6496673226356506 | D accuracy: 61.71875] [G loss: 0.9045023918151855]
2/2 [==============================] - 0s 2ms/step
9930 [D loss: 0.6698790490627289 | D accuracy: 57.03125] [G loss: 0.906700849533081]
2/2 [==============================] - 0s 8ms/step
9931 [D loss: 0.6346114277839661 | D accuracy: 63.28125] [G loss: 0.8538082838058472]
2/2 [==============================] - 0s 5ms/step
993

In [10]:
generator.save("generador_MNIST.h5")

In [11]:
discriminator.save("discriminador_MNIST.h5")